In [12]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io

# Scaled variable
L = 1
W = 0.2

domain = mesh.create_box(MPI.COMM_WORLD, [np.array([0,0,0]), np.array([L, W, W])],
                  [20,6,6], cell_type=mesh.CellType.hexahedron)
V = fem.VectorFunctionSpace(domain, ("CG", 1))

In [13]:
def clamped_boundary(x):
    return np.isclose(x[0], 0)

fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

u_D = np.array([0,0,0], dtype=ScalarType)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [14]:
T = fem.Constant(domain, ScalarType((0, 0, 0)))
ds = ufl.Measure("ds", domain=domain)


In [15]:
# def epsilon(u):
#     return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
# def sigma(u):
#     return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

alpha = 6          # parameter alpha
beta = 1.2         # parameter beta


f_heat = fem.Constant(domain, ScalarType(beta - 2 - 2*alpha))


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# f = fem.Constant(domain, ScalarType((0, 0, -rho*g)))
# a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
# L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds


F_heat = ufl.inner(u,v)*ufl.dx + dt*ufl.dot(grad(u), ufl.grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)


NameError: name 'dt' is not defined

In [9]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

NameError: name 'a' is not defined